# **Analyzing American Baby Name Trends Using SQL**
![hello_my_name_is_cards](https://upload.wikimedia.org/wikipedia/commons/2/26/Hello_My_Name_Is_%2815283079263%29.jpg?20160131002645)

This project uses data provided by the United States Social Security Administration from 1880-2014 to investigate how American baby names have changed over the years. We will be using SQL to analyze what names have stood the test of time.

In [ ]:
#Connecting to SQL Database
import sqlite3
import pandas as pd

conn= sqlite3.connect("datacamp_python.db")

In [ ]:
#Importing CSV file
names = pd.read_csv("/kaggle/input/us-baby-names/NationalNames.csv")
names.to_sql(
            'baby_names',             # Name of the sql table
             conn,                 # sqlite.Connection or sqlalchemy.engine.Engine
             if_exists='replace'
            )

Let's first take a look at what names have appeared in all years of our study period.

In [ ]:
cursor = conn.cursor()
data_sql_1 = pd.read_sql("""
                        SELECT Name, 
                        SUM(Count) AS Sum 
                        FROM baby_names 
                        GROUP BY Name
                        HAVING COUNT(*) = 135
                        ORDER BY sum DESC
                        ;
                        """,
                        conn)
print(data_sql_1)



Wow! It looks like Alexandria is a really popular female name that has stood the test of time. In fact, the Top 5 popular names are all female.

Let's broaden our understanding of the dataset by looking at all names. We shall attempt to categorize each name into a popularity type (Classic, Semi-Classic, Semi-trendy and Trendy) based on how often they show up in our dataset. 

In [ ]:
data_sql_2 = pd.read_sql("""
                        SELECT Name,
                        SUM(Count) AS Sum,
                        CASE WHEN COUNT(*) > 80 THEN 'Classic'
                         WHEN COUNT(*) > 50 THEN 'Semi-classic'
                         WHEN COUNT(*) > 20 THEN 'Semi-trendy'
                        ELSE 'Trendy' END AS Popularity_Type
                        FROM baby_names
                        GROUP BY Name
                        ORDER BY Name
                        ;
                        """,
                        conn)
print(data_sql_2)

Since we didn't really see a lot of traditional male names on the top for our first SQL query, let's filter to only the male names in our dataset and rank them based on the number of babies that have ever been given that name.

In [ ]:
data_sql_3 = pd.read_sql("""
                        SELECT Name,
                        SUM(Count) AS Sum,
                        RANK() OVER (ORDER BY SUM(Count) DESC) AS Name_Rank
                        FROM baby_names 
                        WHERE Gender = 'M'
                        GROUP BY Name
                        LIMIT 10
                        ;
                        """,
                        conn)
print(data_sql_3)

Looks like James and John were really popular names!

Next, let's take a look at the Top 10 popular female names of 2014 that ended in the letter A.

In [ ]:
data_sql_4 = pd.read_sql("""
                        SELECT Name
                        FROM baby_names
                        WHERE Gender = 'F' AND Year = 2014 AND Name LIKE '%a'
                        GROUP BY Name
                        ORDER BY SUM(Count) DESC
                        LIMIT 10
                        ;
                        """,
                        conn)
print(data_sql_4)

Let's now explore the top male names in each year.

In [ ]:
data_sql_6 = pd.read_sql("""
                        SELECT B.Year,
                        A.Name,
                        B.Count
                        FROM baby_names AS A
                        right join (SELECT Year,
                        MAX(Count) AS Count
                        FROM baby_names
                        WHERE Gender = 'M'
                        GROUP BY Year) AS B
                        ON A.Year = B.Year AND A.Count = B.Count
                        ORDER BY B.Year DESC
                        ;
                        """,
                        conn)
print(data_sql_6)

Looks like Noah and Jacob were the top names in 2010-2014. However, if we look at the early 1800 years, John also appears to have been a popular name. 

Now let's see which male name has been number one for the largest number of years

In [ ]:
data_sql_7 = pd.read_sql("""
                        WITH TEMP AS (SELECT B.Year,
                        A.Name,
                        B.Max_Sum
                        FROM baby_names AS A
                        right join (SELECT Year,
                        MAX(Count) AS Max_Sum
                        FROM baby_names
                        WHERE Gender = 'M'
                        GROUP BY Year) AS B
                        ON A.Year = B.Year AND A.Count = B.Max_Sum
                        ORDER BY B.Year DESC)

                        SELECT Name,
                        COUNT(*) AS Count_Top_Name
                        FROM TEMP
                        GROUP BY Name
                        ORDER BY Count_Top_Name DESC
                        ;
                        """,
                        conn)
print(data_sql_7)

Looks like it's a tie between Michael and John, both appearing in the top spot for the largest number of years between 1880-2014.